# __Diffusion Kurtosis Imaging (DKI) analysis notebook__
#### __Last updated on:__ 19/11/2019
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. Multi-shell DWI dataset (minimum two bvalues)
#### 2. DIPY (https://dipy.org/)

### __This script includes:__
#### 1. DKI parametric maps estimation

In [3]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline


#--------------------------------------------------------------
#        Load pre-processed DWI data and brain mask 
#--------------------------------------------------------------
data_path = "/home/User/Diffusion"
file1 = 'eddy_corrected.nii.gz'
path1 = os.path.join(data_path,file1)

if os.path.exists(path1):
        
    print('Reading pre-processed diffusion data')
    img1 = nib.load(path1)
    data = img1.get_data()

    file2 = 'brain_mask.nii.gz'
    path2 = os.path.join(data_path, file2)
    img2 = nib.load(path2)
    mask = img2.get_data()

    fbval = data_path+'/dMRI.bvals'
    fbvec = data_path+'/eddy_corrected.eddy_rotated_bvecs'
    from dipy.io import read_bvals_bvecs
    bvals, bvecs = read_bvals_bvecs(fbval,fbvec)
        
    from dipy.core.gradients import gradient_table
    gtab = gradient_table(bvals, bvecs)

    #--------------------------------------------------------------
    #               Fit diffusion kurtosis model 
    #--------------------------------------------------------------
    print('Fitting diffuison kurtosis model')
    import dipy.reconst.dki as dki
    dkimodel = dki.DiffusionKurtosisModel(gtab)
    dkifit = dkimodel.fit(data, mask)
        
    #--------------------------------------------------------------
    #               Save DKI parametric maps
    #--------------------------------------------------------------
    if not os.path.exists(data_path+'/DKI/'):
        os.mkdir(data_path+'/DKI')
    data_path_saveImage = data_path+'/DKI/'
        
    FA = dkifit.fa
    MD = dkifit.md
    RD = dkifit.rd
    AD = dkifit.ad

    MK = dkifit.mk(0, 3)
    AK = dkifit.ak(0, 3)
    RK = dkifit.rk(0, 3)
        
    nib.save(nib.Nifti1Image(FA, img1.affine), os.path.join(data_path_saveImage,'dki_FA.nii.gz'))
    nib.save(nib.Nifti1Image(MD, img1.affine), os.path.join(data_path_saveImage,'dki_MD.nii.gz'))
    nib.save(nib.Nifti1Image(RD, img1.affine), os.path.join(data_path_saveImage,'dki_RD.nii.gz'))
    nib.save(nib.Nifti1Image(AD, img1.affine), os.path.join(data_path_saveImage,'dki_AD.nii.gz'))
        
    nib.save(nib.Nifti1Image(AK, img1.affine), os.path.join(data_path_saveImage,'AK.nii.gz'))
    nib.save(nib.Nifti1Image(RK, img1.affine), os.path.join(data_path_saveImage,'RK.nii.gz'))
    nib.save(nib.Nifti1Image(MK, img1.affine), os.path.join(data_path_saveImage,'MK.nii.gz'))
        
    print('Processing completed')
    print('Enjoy!!')

Reading pre-processed diffusion data
Fitting diffuison kurtosis model
Processing completed
Enjoy!!
